In [2]:
import os
import json
import pandas as pd
# import altair as alt
# alt.data_transformers.disable_max_rows()
# import geopandas as gpd

## DATA CAN BE SHARED UPON REQUEST ##

# Numbeo

In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
url = 'https://www.numbeo.com/cost-of-living/in/Denver'
page = requests.get(url)

In [5]:
soup = BeautifulSoup(page.text, "html.parser")

In [6]:
alldata = soup.findAll( class_='data_wide_table new_bar_table')

In [7]:
names = []
prices = []
ranges = []

for e in alldata[0].find_all():
    name_ = e.find('td', attrs={'class': None})
    name_2 = e.find('td', class_='tr_highlighted')
    price = e.find('td', class_='priceValue')
    range_ = e.find('td', class_='priceBarTd')

    if name_:
        names.append(name_.text)
        if price:
            prices.append(price.text)
            if range_:
                ranges.append(range_.text.strip())
    elif name_2:
        names.append(name_2.text)
        if price:
            prices.append(price.text)
            if range_:
                ranges.append(range_.text.strip())    


In [8]:
# len(names), len(prices), len(ranges)

In [9]:
numbeo_df = pd.DataFrame([names, prices, ranges]).T
numbeo_df.columns = ['product','price', 'range']

In [10]:
numbeo_df

,product,price,range
0,"Meal, Inexpensive Restaurant",15.52 $,11.00-28.00
1,"Meal for 2 People, Mid-range Restaurant, Three...",66.79 $,50.00-100.00
2,McMeal at McDonalds (or Equivalent Combo Meal),8.00 $,7.00-10.00
3,Domestic Beer (0.5 liter draught),5.75 $,4.00-8.00
4,Imported Beer (0.33 liter bottle),7.50 $,6.00-9.00
5,Cappuccino (regular),4.74 $,3.00-7.00
6,Coke/Pepsi (0.33 liter bottle),1.99 $,1.50-3.00
7,Water (0.33 liter bottle),1.60 $,1.00-2.50
8,"Milk (regular), (1 liter)",0.85 $,0.66-1.19
9,Loaf of Fresh White Bread (500g),2.74 $,1.32-4.96


In [11]:
# %store numbeo_df

Stored 'numbeo_df' (DataFrame)


# Data exploration

## City polygon

In [2]:
PATH = 'Denver/dnvr_co/documenu/geo/'
columns = ['restaurant_name', 'restaurant_phone', 'restaurant_website', 'hours', 'price_range', 'price_range_num', 'restaurant_id', 'cuisines', 'address', 'geo', 'menus', 'last_updated']
restaurants_df = pd.DataFrame(columns=columns)

for file_ in os.listdir(PATH):
    with open(f'{PATH}{file_}') as json_file:
        parsed = json.load(json_file)
        try:
            data = parsed['data']
            for restaurant in data:
                restaurants_df.loc[len(restaurants_df)] = restaurant
        except KeyError:
            pass

In [3]:
restaurants_df.shape, restaurants_df.restaurant_id.nunique()

((595, 12), 595)

In [4]:
# # drop duplicates
# restaurants_df = restaurants_df[~restaurants_df[['restaurant_name', 
#                 'restaurant_phone', 
#                 'restaurant_website', 
#                 'hours', 
#                 'price_range', 
#                 'price_range_num', 
#                 'restaurant_id', 
#                 'last_updated']].duplicated()]

In [5]:
# restaurants_df['lat'] = restaurants_df['geo'].apply(lambda x: x['lat'])
# restaurants_df['lon'] = restaurants_df['geo'].apply(lambda x: x['lon'])

# # restaurants_df['lat'].max(), restaurants_df['lat'].min(), restaurants_df['lon'].max(), restaurants_df['lon'].min()

In [6]:
restaurants_df.head()

,restaurant_name,restaurant_phone,restaurant_website,hours,price_range,price_range_num,restaurant_id,cuisines,address,geo,menus,last_updated
0,Opal,(303) 861-7999,http://www.opalrestaurant.com/,,$,1,39730437104986030,[Japanese],"{'city': 'DENVER', 'state': 'CO', 'postal_code...","{'lat': 39.730437, 'lon': -104.986028}","[{'menu_name': 'Dinner', 'menu_sections': [{'s...",2021-01-05T07:22:35.253Z
1,Wahoo's,(303) 292-0850,,,,0,39747555104984580,[Mexican],"{'city': 'DENVER', 'state': 'CO', 'postal_code...","{'lat': 39.747555, 'lon': -104.984575}","[{'menu_name': 'Main', 'menu_sections': [{'sec...",2021-01-05T07:22:14.325Z
2,Woody Creek Bakery and Cafe,(303) 436-1192,http:///co/denver/640615-woody-creek-bakery-an...,,,0,39748020104994660,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.74802, 'lon': -104.994659}","[{'menu_name': 'Main', 'menu_sections': [{'sec...",2021-01-05T07:21:51.824Z
3,Anise Modern Vietnamese Eatery,(303) 830-3934,http:///co/denver/634259-anise-modern-vietname...,,,0,39730289104986360,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.730289, 'lon': -104.986359}","[{'menu_name': 'Main', 'menu_sections': [{'sec...",2021-01-05T07:21:52.773Z
4,The Avenue Grill,(303) 861-2820,http://www.avenuegrill.com,,$,1,39743088104978904,"[American, American (New)]","{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.743088, 'lon': -104.978905}","[{'menu_name': 'Main', 'menu_sections': [{'sec...",2021-01-05T07:22:37.758Z


### Restaurant Chains

In [7]:
chains = restaurants_df.restaurant_name.value_counts()[restaurants_df.restaurant_name.value_counts()>1]

In [8]:
chains.index

Index(['Starbucks Coffee', 'Subway', 'Starbucks', 'Quiznos Sub',
       'Noodles & Company', 'Chipotle', 'The Delectable Egg', 'Wendy's',
       'Panera Bread', 'Corner Bakery Cafe', 'Pizza House of Colorado',
       'The Curtis Club', 'Potbelly Sandwich Shop', 'Heidi's Brooklyn Deli',
       'Wahoo's', 'Burger King', 'Jamba', 'Wall Street Deli', 'Tokyo Joe's',
       'Spicy Pickle', 'Pho 16th Vietnamese Grill', 'Mad Greens', 'Snarf's',
       'Tryst Lounge', 'The 9th Door', 'Ruth's Chris Steak House'],
      dtype='object')

In [9]:
100*len(chains)/len(restaurants_df) # percent of chains in data

4.369747899159663

#### Subway

In [10]:
subway = restaurants_df[restaurants_df.restaurant_name == 'Subway']

In [11]:
subway

,restaurant_name,restaurant_phone,restaurant_website,hours,price_range,price_range_num,restaurant_id,cuisines,address,geo,menus,last_updated
39,Subway,(303) 436-0486,http://www.subway.com,,,0,39748543104996590,[Sandwiches],"{'city': 'DENVER', 'state': 'CO', 'postal_code...","{'lat': 39.748543, 'lon': -104.996593}","[{'menu_name': 'Main', 'menu_sections': [{'sec...",2021-01-05T07:22:53.131Z
59,Subway,(303) 436-0486,https://restaurants.subway.com/united-states/c...,,,0,39748600104997500,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7486, 'lon': -104.9975}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:22:08.647Z
77,Subway,(303) 534-9923,https://restaurants.subway.com/united-states/c...,,,0,39728200104991100,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7282, 'lon': -104.9911}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:21:53.637Z
192,Subway,(303) 996-0162,https://restaurants.subway.com/united-states/c...,,,0,39747700104977200,[Sandwiches],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7477, 'lon': -104.9772}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:22:39.989Z
203,Subway,(303) 607-9090,https://restaurants.subway.com/united-states/c...,,,0,39745200105005800,[Sandwiches],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7452, 'lon': -105.0058}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:22:39.924Z
297,Subway,(720) 242-9861,https://restaurants.subway.com/united-states/c...,,,0,39740800104990300,[Sandwiches],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7408, 'lon': -104.9903}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:22:53.182Z
304,Subway,(303) 534-5466,https://restaurants.subway.com/united-states/c...,,,0,39738200104993200,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7382, 'lon': -104.9932}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:21:56.296Z
346,Subway,(303) 996-1178,https://restaurants.subway.com/united-states/c...,,,0,39743600104989800,[Sandwiches],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7436, 'lon': -104.9898}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:22:40.236Z
459,Subway,(303) 831-6262,https://restaurants.subway.com/united-states/c...,,,0,39744700104985000,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7447, 'lon': -104.985}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:21:56.318Z
524,Subway,(303) 573-9861,https://restaurants.subway.com/united-states/c...,,,0,39746600104994300,[],"{'city': 'Denver', 'state': 'CO', 'postal_code...","{'lat': 39.7466, 'lon': -104.9943}","[{'menu_name': '', 'menu_sections': [{'section...",2021-01-05T07:21:49.662Z


In [12]:
subway.to_csv('denver_subway.csv', index=False)

### Menu

In [29]:
def menu_to_df(menu):
    menu_df = pd.DataFrame(columns=['name', 'description', 'pricing', 'price', 'section_name'])
    for item in menu[0]['menu_sections']:
        temp = pd.DataFrame(item['menu_items'])
        temp['section_name'] = item['section_name']
        menu_df = menu_df.append( temp)
    menu_df.drop(columns=['pricing'], inplace=True)
    return menu_df

In [32]:
subway['menu_df'] = subway.menus.map(menu_to_df)

<ipython-input-32-790c673fbc21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subway['menu_df'] = subway.menus.map(menu_to_df)


In [34]:
subway['menu_df'].iloc[0]

,name,description,price,section_name
0,Chipotle Southwest Steak & Cheese Signature Wrap,Saddle up with this delicious tomato basil wra...,7.99,Signature Wraps
1,Savory Rotisserie-Style Chicken Caesar Signatu...,Who can resist the classic combination of a de...,7.99,Signature Wraps
2,"Turkey, Bacon & Guacamole Signature Wrap",The name says it all. A delicious Tomato Basil...,7.99,Signature Wraps
0,Black Forest Ham Chopped Salad,The Black Forest Ham has never been better. Lo...,6.48,Salads
1,Chicken & Bacon Ranch Melt Chopped Salad,Saddle and and try the fresh toasted SUBWAY Ch...,7.78,Salads
2,Cold Cut Combo Chopped Salad,Can't decide what kind of meat you want? Get t...,6.48,Salads
3,Italian BMT® Chopped Salad,This all time Italian classic is filled with G...,7.48,Salads
4,Meatball Marinara Chopped Salad,A SUBWAY restaurants favorite. Enjoy Italian s...,6.48,Salads
5,Oven Roasted Chicken Breast Chopped Salad,The Oven Roasted Chicken you love is piled hig...,6.48,Salads
6,Roast Beef Chopped Salad,Piled high with thinly cut slices of lean roas...,7.69,Salads


#### products : SKIP

In [103]:
# df_menus = pd.DataFrame(columns=['name', 'description', 'price', 'section_name', 'restaurant_name'])

# for row in restaurants_df[['restaurant_name','menu_df']].iterrows():
#     restaurant_name = row[1]['restaurant_name']
#     menu = row[1]['menu_df']
# #     print(menu.columns)
#     menu['restaurant_name'] = restaurant_name
#     df_menus = df_menus.append(menu)

In [104]:
# df_menus.shape

(615477, 5)

In [107]:
# df_menus.isna().sum()

name               0
description        0
price              0
section_name       0
restaurant_name    0
dtype: int64

In [108]:
# df_menus[df_menus.description != '']

,name,description,price,section_name,restaurant_name
1,Robertitos,"Carne asada, egg, cheese, potatoes.",5.1,Breakfast Burritos,Los Alejandros
1,Bandido Burrito,"Carne asada, chile, r.lettuce, cheese, enchila...",6.5,Burritos,Los Alejandros
2,Wet Burrito Chicken,"With cheese inside, enchilada salsa and cheese...",5.25,Burritos,Los Alejandros
3,Wet Burrito Beef,"With cheese inside, enchilada salsa and cheese...",5.25,Burritos,Los Alejandros
6,Taco Salad,"Lettuce, Mexican salad, guacamole, sour cream ...",5.65,Burritos,Los Alejandros
...,...,...,...,...,...
2,Eggplant with Tofu,"Chinese Eggplant, tofu, and vegetables, cooked...",12.95,Vegetable Entrees,iWok Chinese Restaurant
0,Sapporo 20.3 fl oz Bottle,Must be 21 to purchase.,5.99,Beer (Must be 21 to Purchase),iWok Chinese Restaurant
1,Sapporo 12 fl oz Bottle,Must be 21 to purchase.,4.5,Beer (Must be 21 to Purchase),iWok Chinese Restaurant
2,Modelo 12 fl oz bottle,Must be 21 to purchase.,3.95,Beer (Must be 21 to Purchase),iWok Chinese Restaurant


In [105]:
# df_menus.section_name.value_counts()

Salads                                           21616
Sides                                            16797
Drinks                                           15538
Beverages                                        13999
All Sandwiches                                   13602
                                                 ...  
Sushi and Oyester Bar - Chilled Grand Seafood        1
Frappe Freeze                                        1
Floats 'N Freezes                                    1
Specialty Sandwiches (Archived)                      1
Breakfast Pandan Waffles                             1
Name: section_name, Length: 18486, dtype: int64

In [106]:
# df_menus[df_menus['section_name'] == 'Beverages'].name.value_counts()

Iced Tea                                  369
Bottled Water                             356
Coffee                                    283
Soda                                      250
Milk                                      246
                                         ... 
Orangina                                    1
Soda (Bottle)                               1
Freshly Brewed Tropical Fruit Iced Tea      1
1501866. Diet Coke                          1
3. Orange Soda                              1
Name: name, Length: 3493, dtype: int64

In [92]:

# def export_menu(restaurant_name, menu):
# #     print(restaurant_name)
#     menu['restaurant_name'] = restaurant_name
#     temp = df_menus.append(menu)
#     print(df_menus)

In [93]:
# temp = restaurants_df[:100].apply(lambda x: export_menu(x['restaurant_name'], x['menu_df']), axis=1)

In [57]:
# df_menus

,name,description,price,selection_name,restaurant_name


### Pricing

In [18]:
restaurants_df['average_price'] = restaurants_df['menu_df'].apply(lambda x: x['price'].mean())

In [19]:
restaurants_df[restaurants_df.average_price>150]['menu_df'].iloc[0]

,name,description,price,section_name
0,Chocolate Dipped Strawberries Box,To make our legendary chocolate dipped strawbe...,39.99,Selections
1,Simply Edible Buquet,Filled with an assortment of fresh fruit favor...,39.99,Selections
2,Pineapple Drip Cakes,Includes pineapple drip cakes featuring fresh ...,49.99,Selections
3,Hand-Dipped White and Semisweet Chocolate Stra...,12 count. This delicious combination includes ...,39.99,Selections
4,Delicious Celebration Dipped Fruit Delight,Make any celebration even more delicious with ...,97.99,Selections
0,Berry Chocolate Bouquet,Combines artfully cut fruits both natural and ...,100.00,Catering Menu
1,Fruit Festival,Combines artfully cut fruits both natural and ...,100.00,Catering Menu
2,Star Celebration,Combines artfully cut fruits both natural and ...,110.00,Catering Menu
3,Brighten their Day Party,"All artfully cut fruits; cantaloupe wedges, gr...",119.00,Catering Menu
4,Always in our Hearts,Combines artfully cut fruits both natural and ...,125.00,Catering Menu


In [20]:
restaurants_df.average_price.describe()

count    7246.000000
mean        8.004075
std         6.680781
min         0.000000
25%         4.952280
50%         7.261553
75%        10.507647
max       158.710714
Name: average_price, dtype: float64

In [21]:
# alt.Chart(restaurants_df[['average_price']]).transform_density(
#     'average_price',
#     as_=['average_price', 'density'],
# ).mark_area().encode(
#     x="average_price:Q",
#     y='density:Q',
# )

# # .mark_line(interpolate='monotone').encode(
# #     x=alt.X('average_price:Q'),
# #     y=alt.Y('count()')
# # )

alt.Chart(...)

In [22]:
stats = restaurants_df.average_price.describe()
def price_point_mapping(price):
    if price <= stats['25%']:
        return '$'
    elif price <= stats['50%']:
        return '$$'
    elif price <= stats['75%']:
        return '$$$'
    else:
        return '$$$$'

In [23]:
restaurants_df['price_point'] = restaurants_df['average_price'].map(price_point_mapping)

In [24]:
restaurants_df['price_point'].value_counts()

$$$     1819
$$$$    1815
$       1812
$$      1811
Name: price_point, dtype: int64

In [25]:
base = alt.Chart(restaurants_df[['lat', 'lon', 'restaurant_name', 'price_point']]).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color='price_point:N'
)

In [26]:
base

alt.Chart(...)

In [27]:
# test
menu_to_df(restaurants_df.menus[0])

,name,description,price,section_name
0,"Potato, Bacon, Eggs & Cheese",,5.2,Breakfast Burritos
1,Robertitos,"Carne asada, egg, cheese, potatoes.",5.1,Breakfast Burritos
2,"Ham & Eggs, Cheese",,5.1,Breakfast Burritos
3,"Mexican Salsa, Eggs & Cheese",,5.1,Breakfast Burritos
4,Chorizo with Eggs,,5.1,Breakfast Burritos
...,...,...,...,...
4,Tortilla Chip with Sour Cream,,4.5,Side Orders
5,4 oz. Guacamole,,1.5,Side Orders
6,Tortilla Chip with Guacamole,,4.5,Side Orders
0,Fountain Drinks,,1.9,Beverages


## Foursquare

In [36]:
PATH = 'Denver/foursquere/'
columns=['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'storeId', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'phrases', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors']
foursquare_df = pd.DataFrame(columns=columns)
for file_ in os.listdir(PATH):
    with open(f'{PATH}{file_}') as json_file:
        data = json.load(json_file)
        foursquare_df.loc[len(foursquare_df)] = data['venue']

In [37]:
foursquare_df.shape, foursquare_df.id.nunique()

((1619, 38), 1619)

In [38]:
foursquare_df.head()

,id,name,contact,location,canonicalUrl,categories,verified,stats,url,price,...,phrases,hours,popular,seasonalHours,defaultHours,pageUpdates,inbox,attributes,bestPhoto,colors
0,3fd66200f964a5201eed1ee3,Bayou Bob's,"{'phone': '3035736828', 'formattedPhone': '(30...","{'address': '1635 Glenarm Pl', 'crossStreet': ...",https://foursquare.com/v/bayou-bobs/3fd66200f9...,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",True,{'tipCount': 30},http://www.bayoubobs.com,"{'tier': 2, 'message': 'Moderate', 'currency':...",...,"[{'phrase': 'great cajun food', 'sample': {'en...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",[],"{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '52b3560b11d2240462dcecf2', 'createdAt'...",{'highlightColor': {'photoId': '52b3560b11d224...
1,3fd66200f964a5205ff11ee3,The Buckhorn Exchange,"{'phone': '3035349505', 'formattedPhone': '(30...","{'address': '1000 Osage St', 'crossStreet': 'a...",https://foursquare.com/v/the-buckhorn-exchange...,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",False,{'tipCount': 86},http://www.buckhorn.com,"{'tier': 3, 'message': 'Expensive', 'currency'...",...,"[{'phrase': 'rocky mountain oysters', 'sample'...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'status': 'Likely open', 'richStatus': {'enti...",[],"{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '50652fc6e4b07e803880e295', 'createdAt'...",{'highlightColor': {'photoId': '50652fc6e4b07e...
2,3fd66200f964a5209feb1ee3,The Park Tavern,"{'phone': '3038327667', 'formattedPhone': '(30...","{'address': '931 E 11th Ave', 'crossStreet': '...",https://foursquare.com/v/the-park-tavern/3fd66...,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,{'tipCount': 52},http://theparktaverndenver.com,"{'tier': 2, 'message': 'Moderate', 'currency':...",...,"[{'phrase': 'bingo on sundays', 'sample': {'en...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",[],"{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '50243ed9e4b0c1330d474a2a', 'createdAt'...",{'highlightColor': {'photoId': '50243ed9e4b0c1...
3,3fd66200f964a520b4ec1ee3,Rio Grande Mexican Restaurant,"{'phone': '3036235432', 'formattedPhone': '(30...","{'address': '1525 Blake St', 'crossStreet': 'b...",https://foursquare.com/v/rio-grande-mexican-re...,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",True,{'tipCount': 96},http://www.riograndemexican.com,"{'tier': 2, 'message': 'Moderate', 'currency':...",...,"[{'phrase': 'rio margarita', 'sample': {'entit...","{'status': 'Open until 10:00 PM', 'richStatus'...","{'isOpen': False, 'isLocalHoliday': False, 'ti...","[{'seasonalRange': 'September 1–September 7', ...","{'status': 'Open until 10:00 PM', 'richStatus'...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '5012edf2e4b0c3be96fbf30b', 'createdAt'...",{'highlightColor': {'photoId': '5012edf2e4b0c3...
4,3fd66200f964a520b9ed1ee3,Charlie Brown's Bar & Grill,"{'phone': '3038601655', 'formattedPhone': '(30...","{'address': '980 N Grant St', 'crossStreet': '...",https://foursquare.com/v/charlie-browns-bar--g...,"[{'id': '4bf58dd8d48988d120941735', 'name': 'K...",True,{'tipCount': 48},http://charliebrownsbarandgrill.com,"{'tier': 2, 'message': 'Moderate', 'currency':...",...,"[{'phrase': 'tab on your birthday', 'sample': ...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",[],"{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [

In [39]:
foursquare_df.columns

Index(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories',
       'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating',
       'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere',
       'specials', 'photos', 'reasons', 'storeId', 'hereNow', 'createdAt',
       'tips', 'shortUrl', 'timeZone', 'listed', 'phrases', 'hours', 'popular',
       'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes',
       'bestPhoto', 'colors'],
      dtype='object')

In [40]:
not_of_interest = ['colors', 'bestPhoto', 'inbox', 'pageUpdates', 'seasonalHours', 'shortUrl', 'timeZone',
                  'createdAt', 'hereNow', 'storeId', 'photos', 'specials', 'beenHere', 'allowMenuUrlEdit',
                  'ratingSignals', 'ratingColor', 'url', 'canonicalUrl', 'contact', 'verified' ]

In [41]:
to_explore = ['attributes', 'popular', 'phrases', 'listed', 'tips', 'reasons', 'stats']

In [42]:
of_interest = list(set(foursquare_df.columns)-set(not_of_interest))

In [43]:
foursquare_df[of_interest].head()

,listed,categories,defaultHours,likes,id,reasons,location,attributes,phrases,dislike,hours,tips,popular,ok,price,name,rating,stats
0,"{'count': 21, 'groups': [{'type': 'others', 'n...","[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 58, 'groups': [{'type': 'others', 'c...",3fd66200f964a5201eed1ee3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'address': '1635 Glenarm Pl', 'crossStreet': ...","{'groups': [{'type': 'price', 'name': 'Price',...","[{'phrase': 'great cajun food', 'sample': {'en...",False,"{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 30, 'groups': [{'type': 'others', 'n...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",False,"{'tier': 2, 'message': 'Moderate', 'currency':...",Bayou Bob's,7.3,{'tipCount': 30}
1,"{'count': 200, 'groups': [{'type': 'others', '...","[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 142, 'groups': [{'type': 'others', '...",3fd66200f964a5205ff11ee3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'address': '1000 Osage St', 'crossStreet': 'a...","{'groups': [{'type': 'price', 'name': 'Price',...","[{'phrase': 'rocky mountain oysters', 'sample'...",False,"{'status': 'Open until 9:00 PM', 'richStatus':...","{'count': 86, 'groups': [{'type': 'others', 'n...","{'status': 'Likely open', 'richStatus': {'enti...",False,"{'tier': 3, 'message': 'Expensive', 'currency'...",The Buckhorn Exchange,7.9,{'tipCount': 86}
2,"{'count': 27, 'groups': [{'type': 'others', 'n...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 70, 'groups': [{'type': 'others', 'c...",3fd66200f964a5209feb1ee3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'address': '931 E 11th Ave', 'crossStreet': '...","{'groups': [{'type': 'price', 'name': 'Price',...","[{'phrase': 'bingo on sundays', 'sample': {'en...",False,"{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 52, 'groups': [{'type': 'others', 'n...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",False,"{'tier': 2, 'message': 'Moderate', 'currency':...",The Park Tavern,7.3,{'tipCount': 52}
3,"{'count': 71, 'groups': [{'type': 'others', 'n...","[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...","{'status': 'Open until 10:00 PM', 'richStatus'...","{'count': 218, 'groups': [{'type': 'others', '...",3fd66200f964a520b4ec1ee3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'address': '1525 Blake St', 'crossStreet': 'b...","{'groups': [{'type': 'price', 'name': 'Price',...","[{'phrase': 'rio margarita', 'sample': {'entit...",False,"{'status': 'Open until 10:00 PM', 'richStatus'...","{'count': 96, 'groups': [{'type': 'others', 'n...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",False,"{'tier': 2, 'message': 'Moderate', 'currency':...",Rio Grande Mexican Restaurant,7.5,{'tipCount': 96}
4,"{'count': 55, 'groups': [{'type': 'others', 'n...","[{'id': '4bf58dd8d48988d120941735', 'name': 'K...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 106, 'groups': [{'type': 'others', '...",3fd66200f964a520b9ed1ee3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'address': '980 N Grant St', 'crossStreet': '...","{'groups': [{'type': 'price', 'name': 'Price',...","[{'phrase': 'tab on your birthday', 'sample': ...",False,"{'status': 'Open until 2:00 AM', 'richStatus':...","{'count': 48, 'groups': [{'type': 'others', 'n...","{'isOpen': False, 'isLocalHoliday': False, 'ti...",False,"{'tier': 2, 'message': 'Moderate', 'currency':...",Charlie Brown's Bar & Grill,7.7,{'tipCount': 48}


In [44]:
foursquare_df['lat'] = foursquare_df['location'].apply(lambda x: x['lat'])
foursquare_df['lon'] = foursquare_df['location'].apply(lambda x: x['lng'])
# foursquare_df['address'] = foursquare_df['location'].apply(lambda x: try: x['address'] except '')

In [45]:
foursquare_df.drop(columns=not_of_interest)

,id,name,location,categories,stats,price,likes,dislike,ok,rating,reasons,tips,listed,phrases,hours,popular,defaultHours,attributes,lat,lon
0,3fd66200f964a5201eed1ee3,Bayou Bob's,"{'address': '1635 Glenarm Pl', 'crossStreet': ...","[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",{'tipCount': 30},"{'tier': 2, 'message': 'Moderate', 'currency':...","{'count': 58, 'groups': [{'type': 'others', 'c...",False,False,7.3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'count': 30, 'groups': [{'type': 'others', 'n...","{'count': 21, 'groups': [{'type': 'others', 'n...","[{'phrase': 'great cajun food', 'sample': {'en...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'groups': [{'type': 'price', 'name': 'Price',...",39.744275,-104.989910
1,3fd66200f964a5205ff11ee3,The Buckhorn Exchange,"{'address': '1000 Osage St', 'crossStreet': 'a...","[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",{'tipCount': 86},"{'tier': 3, 'message': 'Expensive', 'currency'...","{'count': 142, 'groups': [{'type': 'others', '...",False,False,7.9,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'count': 86, 'groups': [{'type': 'others', 'n...","{'count': 200, 'groups': [{'type': 'others', '...","[{'phrase': 'rocky mountain oysters', 'sample'...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'status': 'Likely open', 'richStatus': {'enti...","{'status': 'Open until 9:00 PM', 'richStatus':...","{'groups': [{'type': 'price', 'name': 'Price',...",39.732205,-105.005067
2,3fd66200f964a5209feb1ee3,The Park Tavern,"{'address': '931 E 11th Ave', 'crossStreet': '...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",{'tipCount': 52},"{'tier': 2, 'message': 'Moderate', 'currency':...","{'count': 70, 'groups': [{'type': 'others', 'c...",False,False,7.3,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'count': 52, 'groups': [{'type': 'others', 'n...","{'count': 27, 'groups': [{'type': 'others', 'n...","[{'phrase': 'bingo on sundays', 'sample': {'en...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'groups': [{'type': 'price', 'name': 'Price',...",39.733835,-104.975690
3,3fd66200f964a520b4ec1ee3,Rio Grande Mexican Restaurant,"{'address': '1525 Blake St', 'crossStreet': 'b...","[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",{'tipCount': 96},"{'tier': 2, 'message': 'Moderate', 'currency':...","{'count': 218, 'groups': [{'type': 'others', '...",False,False,7.5,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'count': 96, 'groups': [{'type': 'others', 'n...","{'count': 71, 'groups': [{'type': 'others', 'n...","[{'phrase': 'rio margarita', 'sample': {'entit...","{'status': 'Open until 10:00 PM', 'richStatus'...","{'isOpen': False, 'isLocalHoliday': False, 'ti...","{'status': 'Open until 10:00 PM', 'richStatus'...","{'groups': [{'type': 'price', 'name': 'Price',...",39.750223,-105.000036
4,3fd66200f964a520b9ed1ee3,Charlie Brown's Bar & Grill,"{'address': '980 N Grant St', 'crossStreet': '...","[{'id': '4bf58dd8d48988d120941735', 'name': 'K...",{'tipCount': 48},"{'tier': 2, 'message': 'Moderate', 'currency':...","{'count': 106, 'groups': [{'type': 'others', '...",False,False,7.7,"{'count': 1, 'items': [{'summary': 'Lots of pe...","{'count': 48, 'groups': [{'type': 'others', 'n...","{'count': 55, 'groups': [{'type': 'others', 'n...","[{'phrase': 'tab on your birthday', 'sample': ...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'isOpen': False, 'isLocalHoliday': False, 'ti...","{'status': 'Open until 2:00 AM', 'richStatus':...","{'groups': [{'type': 'price', 'name': 'Price',...",39.731927,-104.983316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,5f51984aa77b4b141eef164c,Buffalo Bill's Wings & Things,"{'lat': 39.739857, 'lng': -104.98073, 'labeled...","[{'id': '4bf58dd8d48988d14

In [46]:
# intersections by name with city polygon
len((set(foursquare_df.name)).intersection(set(restaurants_df.restaurant_name)))

196

In [49]:
foursquare_df[foursquare_df.name == 'Subway']

,id,name,contact,location,canonicalUrl,categories,verified,stats,url,price,...,popular,seasonalHours,defaultHours,pageUpdates,inbox,attributes,bestPhoto,colors,lat,lon
124,4a98276ff964a520b42a20e3,Subway,"{'phone': '3035739861', 'formattedPhone': '(30...","{'address': '904 16th St Unit C', 'crossStreet...",https://foursquare.com/v/subway/4a98276ff964a5...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 5},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,"{'status': 'Likely open', 'richStatus': {'enti...","[{'seasonalRange': 'September 28–October 28', ...",NaN,"{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '5720f635498ec1f0451d2b63', 'createdAt'...",{'highlightColor': {'photoId': '5720f635498ec1...,39.746573,-104.994315
199,4b5df8e3f964a520dd7629e3,Subway,"{'phone': '3039961178', 'formattedPhone': '(30...","{'address': '409 16th St', 'crossStreet': 'nr ...",https://foursquare.com/v/subway/4b5df8e3f964a5...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 10},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,"{'status': 'Likely open', 'richStatus': {'enti...",[],"{'status': 'Open until 7:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '5338980c498ea66c3d23439c', 'createdAt'...",{'highlightColor': {'photoId': '5338980c498ea6...,39.743579,-104.989778
360,4c53296da724e21edda150f4,Subway,"{'phone': '3034360486', 'formattedPhone': '(30...","{'address': '1512 Larimer St Unit R19', 'lat':...",https://foursquare.com/v/subway/4c53296da724e2...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 4},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,"{'status': 'Likely open', 'richStatus': {'enti...",[],"{'status': 'Open until 7:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '4f91b184e4b0e03b6694ed02', 'createdAt'...",NaN,39.748456,-104.997683
384,4c76772066be6dcb2327c30f,Subway,"{'phone': '7202429861', 'formattedPhone': '(72...","{'address': '201 W Colfax Ave', 'lat': 39.7408...",https://foursquare.com/v/subway/4c76772066be6d...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 0},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,NaN,[],"{'status': 'Open until 3:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...","{'id': '519911fb498e7ef128d5c29c', 'createdAt'...",NaN,39.740806,-104.990253
854,4fbd38e2e4b0ac6570f3c7c5,Subway,"{'phone': '3036079090', 'formattedPhone': '(30...","{'address': '900 Auraria Pkwy Unit 101B', 'cro...",https://foursquare.com/v/subway/4fbd38e2e4b0ac...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 0},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,NaN,"[{'seasonalRange': 'September 28–October 28', ...","{'status': 'Open until 5:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name': 'Price',...",NaN,NaN,39.745236,-105.005800
1118,526967df498eefeefc84242a,Subway,"{'phone': '3039535384', 'formattedPhone': '(30...","{'address': '1725 Champa St Ste 101', 'lat': 3...",https://foursquare.com/v/subway/526967df498eef...,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,{'tipCount': 0},https://restaurants.subway.com/united-states/c...,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",...,NaN,[],"{'status': 'Open until 7:00 PM', 'richStatus':...","{'count': 0, 'items': []}","{'count': 0, 'items': []}","{'groups': [{'type': 'price', 'name':

## Yelp

In [50]:
PATH = 'Denver/yelp/'
columns=['id', 'alias', 'name', 'image_url', 'is_claimed', 'is_closed', 'url', 'phone', 'display_phone', 'review_count', 'categories', 'rating', 'location', 'coordinates', 'photos', 'price', 'hours', 'transactions', 'reviews']
yelp_df = pd.DataFrame(columns=columns)
for file_ in os.listdir(PATH):
    with open(f'{PATH}{file_}') as json_file:
        data = json.load(json_file)
        yelp_df.loc[len(yelp_df)] = data

In [51]:
columns_of_interest = ['id', 'name', 'is_closed', 'review_count', 'categories', 'rating',
       'location', 'coordinates', 'price', 'hours', 'transactions',
       'reviews']

In [57]:
yelp_df.shape

(1889, 19)

In [52]:
yelp_df[columns_of_interest].head()

,id,name,is_closed,review_count,categories,rating,location,coordinates,price,hours,transactions,reviews
0,-0qRyIVI5FZItFInPjAX9Q,Russell's Stores,False,1,"[{'alias': 'stationery', 'title': 'Cards & Sta...",3.0,"{'address1': '1740 Broadway', 'address2': '', ...","{'latitude': 39.7440567016602, 'longitude': -1...",$,NaN,[],"[{'id': 'iu9sN6DgY4xVKLscYETAqQ', 'url': 'http..."
1,-1UE1ea2LgbfrYnFmlxeYQ,Seattle Fish Company,False,2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'address1': '6211 E 42nd Ave', 'address2': ''...","{'latitude': 39.77487, 'longitude': -104.91568}",NaN,NaN,[],"[{'id': 'hc7-N7P_cPlk2wUJIPw6GQ', 'url': 'http..."
2,-2ekvh8JjVmzY95W1Ie8yA,PF Donut Shop,False,16,"[{'alias': 'donuts', 'title': 'Donuts'}]",3.5,"{'address1': '2855 York St', 'address2': '', '...","{'latitude': 39.758058, 'longitude': -104.960113}",$,NaN,[delivery],"[{'id': 'z073gjVJeVdN4o_9KLHN5g', 'url': 'http..."
3,-4vU2uaWHgKpDKdYiXDIGA,The Delectable Plate,False,8,"[{'alias': 'markets', 'title': 'Fruits & Veggi...",1.0,"{'address1': '', 'address2': '', 'address3': '...","{'latitude': 39.7400093078613, 'longitude': -1...",$$,NaN,[],"[{'id': 'wp9tedg3i7_Z7A-uHu7f5g', 'url': 'http..."
4,-5iF6s8wJbGd9R2CVd5bvg,Walter's303 Lowry Pizzeria & Publik House,False,53,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'address1': '200 Quebec St', 'address2': 'Ste...","{'latitude': 39.7201329769283, 'longitude': -1...",$$,"[{'open': [{'is_overnight': False, 'start': '1...","[delivery, pickup]","[{'id': 'os_w-zs2fdMvqsZH3zVGDA', 'url': 'http..."


In [56]:
yelp_df[yelp_df.name == 'Subway']

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,categories,rating,location,coordinates,photos,price,hours,transactions,reviews
1826,zG0xM-KRFnvhTlhtWyxNNA,subway-denver-25,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/8yeJmC...,True,False,https://www.yelp.com/biz/subway-denver-25?adju...,+13034360486,(303) 436-0486,17,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'address1': '1512 Larimer St', 'address2': ''...","{'latitude': 39.748796, 'longitude': -104.99728}",[https://s3-media2.fl.yelpcdn.com/bphoto/8yeJm...,$,"[{'open': [{'is_overnight': False, 'start': '0...","[pickup, delivery]","[{'id': '6T9kM4j0VQYd8PMjubw9eQ', 'url': 'http..."


In [133]:
# len(set(yelp_df.name).intersection(set(foursquare_df.name)))
# len(set(yelp_df.name).intersection(set(restaurants_df.restaurant_name)))